In [689]:
import requests
import polars as pl
import json
import os
from datetime import datetime,timezone
from copy import copy


pl.Config.set_tbl_rows(10)
pl.Config.set_tbl_cols(100)
pl.Config.set_tbl_width_chars=10000

In [ ]:
ctt = pl.read_csv("coins_to_track.csv")
','.join(ctt['Symbol'])

In [3]:
cmc_key = "57eccd1c-b1de-4cff-8b6a-b4206ffded67"

headers = {
    "X-CMC_PRO_API_KEY": cmc_key,
    "Content-Type": "application/json"
}

parameters = {
    # "symbol": ','.join(ctt['Symbol']),
    "symbol": 'BTC',
    "skip_invalid": "false"
}

# session = requests.Session()
# session.headers.update(headers)

x = json.loads(session.get(f"https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest", params = parameters).text)

In [780]:
def cmc_extract(cmc_endpoint_url: str, headers: dict, parameters: dict):

    """
    returns: api response generator 
    """

    # coins_to_track = pl.read_csv("coins_to_track.csv")
    # symbols = ','.join(ctt['Symbol'])

    # parameters.update(symbol=symbols)
    session = requests.Session()
    session.headers.update(headers)

    responses = json.loads(session.get(cmc_endpoint_url, params = parameters).text)

    return responses




In [ ]:
coins_to_track = pl.read_csv("coins_to_track.csv")
symbols = ','.join(coins_to_track['Symbol'][:])
parameters = {}
parameters.update(symbol=symbols)

print(parameters)

In [ ]:
def get_quotes(cmc_endpoint_url: str, headers: dict, parameters: dict, csv_write_path: str) -> None:

    responses = cmc_extract(cmc_endpoint_url = cmc_endpoint_url, headers = headers, parameters = parameters)
    
    ts = datetime.now(timezone.utc)

    returned_data = responses["data"]
    returned_symbols = list(returned_data.keys())

    # list comp to build list of LazyFrames (Lazy evaluated DataFrame) with quote data for each symbol
    # and adding symbol identifier, IsTopCurrency, and LoadedWhen columns to each chunk
    quote_lfs = [
        pl.LazyFrame(data["quote"]["USD"]).with_columns(
                pl.lit(symbol).alias("symbol"),
                pl.lit(int(returned_data[symbol]["cmc_rank"])<=10).alias("IsTopCurrency"),
                pl.lit(ts.strftime("%Y-%m-%dT%H:%M:%SZ")).alias("LoadedWhen")
            ) 
        for symbol,data in returned_data.items()
    ]

    # concat quote dfs to one single LazyFrame
    lf = pl.concat(quote_dfs, how = 'vertical_relaxed')

    # materialize to DataFrame and write_data to local path
    lf.collect().write_csv(file = fr"{csv_write_path}/{ts.strftime('%Y%m%dT%H%M%S')}_quotes.csv")

    return None

get_quotes( "https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest",headers=headers, parameters={"symbol":symbols, "skip_invalid": "true"}, csv_write_path = "src/extracts/quotes")

# QUOTES LATEST

In [ ]:
# coins_to_track = pl.read_csv("coins_to_track.csv")
# symbols = ','.join(ctt['Symbol'][:])
# parameters = {}
# parameters.update(symbol=symbols)

# print(parameters)

responses = cmc_extract(cmc_endpoint_url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest", headers = headers, parameters = parameters)

returned_data = responses["data"]
returned_symbols = list(returned_data.keys())

quote_dfs = [pl.LazyFrame(returned_data[symbol]["quote"]["USD"]).with_columns(pl.lit(symbol).alias("symbol"), pl.lit(int(returned_data[symbol]["cmc_rank"])<=10).alias("IsTopCurrency")) for symbol in returned_symbols]
df = pl.concat(quote_dfs, how = 'vertical_relaxed')


ts = datetime.now(timezone.utc)
df = df.with_columns(pl.lit(ts.strftime("%Y-%m-%dT%H:%M:%SZ")).alias("LoadedWhen"))
df.collect().write_csv(file = fr"src/extracts/{ts.strftime('%Y%m%dT%H%M%S')}_quotes.csv")




# METADATA

In [797]:
responses2 = cmc_extract(cmc_endpoint_url = "https://pro-api.coinmarketcap.com/v2/cryptocurrency/info", headers = headers, parameters = {"symbol":symbols, "skip_invalid": "true"})

returned_data2 = responses2["data"]
returned_symbols2 = list(returned_data2.keys())

In [ ]:
# pl.json_normalize(returned_data2, strict = False, max_level = 5)
# pl.json_normalize(returned_data2["AAVE"])

# meta_dfs = [pl.json_normalize(data, strict = False, max_level = 5) for data in returned_data2.values()]

# for i in meta_dfs:
#     print(i.shape)


meta_dfs = [pl.json_normalize(j, strict = False, max_level = 10) for i,j in returned_data2.items()]

meta_dfs = [df for df in meta_dfs if df.shape != (0,0)]

max_col_set = set()
for df in meta_dfs:
    max_col_set = max_col_set.union(set(df.columns))
    # print(df.shape)


def add_missing_null_columns(df, col_set):

    def go_add_col(df, col_set):
        
        if bool(col_set):
            x = col_set.pop()

            go_add_col(df.insert_column(1,pl.Series(f"{x}",[None]*df.shape[0])), col_set)

        else:
            return df

    if df.shape != (0,0):
        go_add_col(df, col_set)
    else:
        raise Exception("EMPTY DATAFRAME")

for df in meta_dfs:
    add_missing_null_columns(df,max_col_set - set(df.columns))
    # print(set(df.columns))

meta_df = pl.concat([df.select(max_col_set) for df in meta_dfs if df.shape != (0,0)], how = "vertical_relaxed")

meta_df = \
    meta_df.select(
        "id",
        "name",
        "symbol",
        "category",
        "slug",
        "logo",
        "description",
        "date_added",
        "date_launched",
        "notice",
        "tags",
        "platform",
        "platform.id",
        "platform.name",
        "platform.symbol",
        "platform.slug",
        "platform.token_address",
        "self_reported_circulating_supply",
        "self_reported_market_cap",
        "self_reported_tags",
        "infinite_supply",
        "urls.website",
        "urls.technical_doc",
        "urls.explorer",
        "urls.source_code",
        "urls.message_board",
        "urls.chat",
        "urls.announcement",
        "urls.reddit",
        "urls.twitter",
        "contract_address"
    )


# for i in meta_df["contract_address"]:
#     print(pl.json_normalize(i))

# meta_df = \
#     meta_df.with_columns(

#         pl.col("contract_address").list.first().struct.rename_fields(["contract_address","contract_address.platform"])

#     ).unnest("contract_address").with_columns(

#         pl.col("contract_address.platform").struct.rename_fields(["contract_address.platform.name","contract_address.platform.coin"])

#     ).unnest("contract_address.platform").with_columns(

#         pl.col("contract_address.platform.coin").struct.rename_fields(["contract_address.platform.coin.id","contract_address.platform.coin.name","contract_address.platform.coin.symbol","contract_address.platform.coin.slug"])

#     ).unnest("contract_address.platform.coin")

# meta_df

In [ ]:
meta_dfs = [pl.from_records(returned_data2[symbol]) for symbol in returned_data2.keys()]
meta_df = pl.concat([df for df in meta_dfs if df.shape != (0,0)], how = "vertical_relaxed")

meta_df = \
    meta_df.with_columns(

        pl.col("contract_address").list.first().struct.rename_fields(["contract_address","contract_address.platform"])

    ).unnest("contract_address").with_columns(

        pl.col("contract_address.platform").struct.rename_fields(["contract_address.platform.name","contract_address.platform.coin"])

    ).unnest("contract_address.platform").with_columns(

        pl.col("contract_address.platform.coin").struct.rename_fields(["contract_address.platform.coin.id","contract_address.platform.coin.name","contract_address.platform.coin.symbol","contract_address.platform.coin.slug"])

    ).unnest("contract_address.platform.coin").with_columns(

        pl.col("platform").struct.rename_fields(["platform.id","platform.name","platform.slug","platform.symbol","platform.token_address"])

    ).unnest(["platform","urls"])

ts = datetime.now(timezone.utc)
meta_df = meta_df.with_columns(pl.lit(ts.strftime("%Y-%m-%dT%H:%M:%SZ")).alias("LoadedWhen"))


for column,type in dict(zip(meta_df.columns,meta_df.dtypes)).items():

    if type == pl.List:
        meta_df = meta_df.with_columns(pl.col(column).list.join("|"))


meta_df.write_csv(file = fr"src/extracts/metadata/{ts.strftime('%Y%m%dT%H%M%S')}_metadata.csv")

meta_df

In [ ]:
def get_metadata(cmc_endpoint_url: str, headers: dict, parameters: dict, csv_write_path: str) -> None:

    responses = cmc_extract(cmc_endpoint_url = cmc_endpoint_url, headers = headers, parameters = parameters)
    
    ts = datetime.now(timezone.utc)

    returned_data = responses["data"]
    returned_symbols = list(returned_data.keys())

    meta_dfs = [pl.from_records(data) for symbol,data in returned_data.items()]
    meta_df = pl.concat([df for df in meta_dfs if df.shape != (0,0)], how = "vertical_relaxed")


    # Unnesting contract_address, platform, and urls nested fields
    meta_df = \
        meta_df.with_columns(

            pl.col("contract_address").list.first().struct.rename_fields(["contract_address","contract_address.platform"])

        ).unnest("contract_address").with_columns(

            pl.col("contract_address.platform").struct.rename_fields(["contract_address.platform.name","contract_address.platform.coin"])

        ).unnest("contract_address.platform").with_columns(

            pl.col("contract_address.platform.coin").struct.rename_fields(["contract_address.platform.coin.id","contract_address.platform.coin.name","contract_address.platform.coin.symbol","contract_address.platform.coin.slug"])

        ).unnest("contract_address.platform.coin").with_columns(

            pl.col("platform").struct.rename_fields(["platform.id","platform.name","platform.slug","platform.symbol","platform.token_address"])

        ).unnest(["platform","urls"])

    
    meta_df = meta_df.with_columns(pl.lit(ts.strftime("%Y-%m-%dT%H:%M:%SZ")).alias("LoadedWhen"))


    for column,type in dict(zip(meta_df.columns,meta_df.dtypes)).items():
        # Convert fields of type List[] to pipe delimited strings so we can write to csv
        if type == pl.List:
            meta_df = meta_df.with_columns(pl.col(column).list.join("|"))


    meta_df.write_csv(file = fr"src/extracts/metadata/{ts.strftime('%Y%m%dT%H%M%S')}_metadata.csv")

    return meta_df

get_metadata(cmc_endpoint_url = "https://pro-api.coinmarketcap.com/v2/cryptocurrency/info", headers = headers, parameters = {"symbol":symbols, "skip_invalid": "true"}, csv_write_path = "src/extracts/metadata")

# Mapping Data

In [367]:
responses = cmc_extract(server_url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/map", headers = headers, parameters = {})

returned_data = responses["data"]
# returned_symbols = list(returned_data.keys())

map_dfs = [pl.from_records(returned_data).sort(by = 'symbol').filter(pl.col('symbol').eq(symbol)) for symbol in coins_to_track['Symbol']]

map_df = pl.concat(map_dfs, how = 'vertical_relaxed')
map_df = map_df.with_columns((pl.col("rank").cast(pl.Int64) <= 10).alias("IsTopCurrency"))

ts = datetime.now(timezone.utc)
map_df = map_df.with_columns(pl.lit(ts.strftime("%Y-%m-%dT%H:%M:%SZ")).alias("LoadedWhen"))
map_df.select(
    "name",
    "symbol",
    "slug",
    "is_active",
    "status",
    "first_historical_data",
    "last_historical_data",
    "IsTopCurrency"
    ).write_csv(file = fr"src/extracts/map/{ts.strftime('%Y%m%dT%H%M%S')}_map.csv")